<a href="https://colab.research.google.com/github/maderamel/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_clean = application_df.drop(columns = ["EIN", "NAME"])

In [3]:
# Determine the number of unique values in each column.
application_df_clean.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df_clean.APPLICATION_TYPE.value_counts()

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df_clean.CLASSIFICATION.value_counts()

classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
  application_df_clean['CLASSIFICATION'] = application_df_clean['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
application_df_clean['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_clean = pd.get_dummies(application_df_clean, columns=['CLASSIFICATION'])

application_df_clean.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other
0,T10,Independent,ProductDev,Association,1,0,N,5000,1,1,0,0,0,0,0
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,0,0,1,0,0,0
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0,0,0,0,0,1,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,0,0,1,0,0,0
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,1,0,0,0,0,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df_clean["IS_SUCCESSFUL"].values
X = application_df_clean.drop(["APPLICATION_TYPE", "AFFILIATION",	"USE_CASE",	"ORGANIZATION",	"INCOME_AMT",	"SPECIAL_CONSIDERATIONS"],1).values
0
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


<ipython-input-10-9ed851805470>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df_clean.drop(["APPLICATION_TYPE", "AFFILIATION",	"USE_CASE",	"ORGANIZATION",	"INCOME_AMT",	"SPECIAL_CONSIDERATIONS"],1).values


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 80        
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 0.2474 - accuracy: 0.8887
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.0105 - accuracy: 0.9999
Epoch 3/100
804/804 [==============================] - 4s 6ms/step - loss: 0.0016 - accuracy: 0.9999
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 5.1953e-04 - accuracy: 1.0000
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 2.6510e-04 - accuracy: 1.0000
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 2.6045e-04 - accuracy: 1.0000
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 2.6699e-04 - accuracy: 1.0000
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 4.7102e-05 - accuracy: 1.0000
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 6.8190e-05 - accuracy: 1.0000
Epoch 10/100
804/804 [==============================] - 2s 3ms/step

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 1.1929e-09 - accuracy: 1.0000 - 527ms/epoch - 2ms/step
Loss: 1.1929371934726873e-09, Accuracy: 1.0


In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                100       
                                                                 
 dense_4 (Dense)             (None, 2)                 22        
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 0.3559 - accuracy: 0.8722
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.0130 - accuracy: 0.9997
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.0024 - accuracy: 0.9999
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 8.5511e-04 - accuracy: 1.0000
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.0013 - accuracy: 0.9999
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 2.3513e-04 - accuracy: 1.0000
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 1.4221e-04 - accuracy: 1.0000
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 8.5612e-05 - accuracy: 1.0000
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 4.9229e-05 - accuracy: 1.0000
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 2.944

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               1000      
                                                                 
 dense_7 (Dense)             (None, 2)                 202       
                                                                 
 dense_8 (Dense)             (None, 1)                 3         
                                                                 
Total params: 1,205
Trainable params: 1,205
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.2581 - accuracy: 0.9880
Epoch 2/200
804/804 [==============================] - 2s 3ms/step - loss: 0.1392 - accuracy: 0.9998
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.0855 - accuracy: 0.9998
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.0548 - accuracy: 0.9998
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.0367 - accuracy: 0.9998
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.0250 - accuracy: 0.9999
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.0176 - accuracy: 0.9998
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.0125 - accuracy: 0.9999
Epoch 9/200
804/804 [==============================] - 2s 3ms/step - loss: 0.0121 - accuracy: 0.9998
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.0066 - accura

In [31]:
# Export our model to HDF5 file
from google.colab import drive

#drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/nn.h5"

nn.save(file_path)